<a href="https://colab.research.google.com/github/anushkapatwa/CSCI-167/blob/main/Notebooks/Chap12/12_1_Self_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook 12.1: Self Attention**

This notebook builds a self-attention mechanism from scratch, as discussed in section 12.2 of the book.

Work through the cells below, running each cell in turn. In various places you will see the words "TODO". Follow the instructions at these places and make predictions about what is going to happen or write code to complete the functions.

Contact me at udlbookmail@gmail.com if you find any mistakes or have any suggestions.



In [1]:
import numpy as np
import matplotlib.pyplot as plt

The self-attention mechanism maps $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ and returns $N$ outputs $\mathbf{x}'_{n}\in \mathbb{R}^{D}$.  



In [2]:
# Set seed so we get the same random numbers
np.random.seed(3)
# Number of inputs
N = 3
# Number of dimensions of each input
D = 4
# Create an empty list
all_x = []
# Create elements x_n and append to list
for n in range(N):
  all_x.append(np.random.normal(size=(D,1)))
# Print out the list
print(all_x)


[array([[ 1.78862847],
       [ 0.43650985],
       [ 0.09649747],
       [-1.8634927 ]]), array([[-0.2773882 ],
       [-0.35475898],
       [-0.08274148],
       [-0.62700068]]), array([[-0.04381817],
       [-0.47721803],
       [-1.31386475],
       [ 0.88462238]])]


We'll also need the weights and biases for the keys, queries, and values (equations 12.2 and 12.4)

In [3]:
# Set seed so we get the same random numbers
np.random.seed(0)

# Choose random values for the parameters
omega_q = np.random.normal(size=(D,D))
omega_k = np.random.normal(size=(D,D))
omega_v = np.random.normal(size=(D,D))
beta_q = np.random.normal(size=(D,1))
beta_k = np.random.normal(size=(D,1))
beta_v = np.random.normal(size=(D,1))

Now let's compute the queries, keys, and values for each input

In [7]:
all_queries = []
all_keys = []
all_values = []

for x in all_x:
    # Reshape x to (1, D) so we can multiply
    x = x.reshape(1, -1)

    query = x @ beta_q   # (1,1)
    key   = x @ beta_k
    value = x @ beta_v

    all_queries.append(query)
    all_keys.append(key)
    all_values.append(value)


We'll need a softmax function (equation 12.5) -- here, it will take a list of arbitrary numbers and return a list where the elements are non-negative and sum to one


In [8]:
def softmax(items_in):
    # Convert to numpy array (safe if already one)
    x = np.array(items_in, dtype=float)

    # Subtract max value for numerical stability
    x = x - np.max(x)

    # Compute exponentials
    exp_x = np.exp(x)

    # Normalize by sum
    items_out = exp_x / np.sum(exp_x)

    return items_out


Now compute the self attention values:

In [9]:
# Create empty list for output
all_x_prime = []

# For each output
for n in range(N):
    # Create list for dot products of query_n with all keys
    all_km_qn = []

    # Get query vector for this index
    q_n = all_queries[n]  # shape (1,1)

    # Compute the dot products
    for k_m in all_keys:
        dot_product = float(q_n * k_m)  # both are (1,1), so multiply and convert to scalar
        all_km_qn.append(dot_product)

    # Softmax to get attention weights
    attention = softmax(all_km_qn)
    print("Attentions for output ", n)
    print(attention)

    # Weighted sum of values (Equation 12.3)
    x_prime = np.zeros((D,1))
    for weight, v_m in zip(attention, all_values):
        x_prime += weight * v_m  # weighted value sum

    all_x_prime.append(x_prime)


Attentions for output  0
[9.99924599e-01 7.05581441e-05 4.84236921e-06]
Attentions for output  1
[0.15165763 0.37123719 0.47710518]
Attentions for output  2
[0.00323997 0.23138132 0.76537871]


/tmp/ipython-input-876888328.py:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  dot_product = float(q_n * k_m)  # both are (1,1), so multiply and convert to scalar


Now let's compute the same thing, but using matrix calculations.  We'll store the $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ in the columns of a $D\times N$ matrix, using equations 12.6 and 12.7/8.

Note:  The book uses column vectors (for compatibility with the rest of the text), but in the wider literature it is more normal to store the inputs in the rows of a matrix;  in this case, the computation is the same, but all the matrices are transposed and the operations proceed in the reverse order.

In [10]:
# Define softmax operation that works independently on each column
def softmax_cols(data_in):
  # Exponentiate all of the values
  exp_values = np.exp(data_in) ;
  # Sum over columns
  denom = np.sum(exp_values, axis = 0);
  # Replicate denominator to N rows
  denom = np.matmul(np.ones((data_in.shape[0],1)), denom[np.newaxis,:])
  # Compute softmax
  softmax = exp_values / denom
  # return the answer
  return softmax

In [11]:
def self_attention(X, omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):

    # 1. Compute Queries, Keys, Values:
    # Q = X * ω_q + β_q
    Q = X @ omega_q + beta_q.T        # (N, D)
    K = X @ omega_k + beta_k.T        # (N, D)
    V = X @ omega_v + beta_v.T        # (N, D)

    # 2. Compute dot products (similarity scores)
    # QK^T gives an (N x N) matrix
    scores = Q @ K.T                  # (N, N)

    # 3. Apply softmax row-wise to get attention weights
    # We subtract max for numerical stability
    scores = scores - np.max(scores, axis=1, keepdims=True)
    attention = np.exp(scores)
    attention = attention / np.sum(attention, axis=1, keepdims=True)   # (N, N)

    # 4. Compute weighted sum of values
    X_prime = attention @ V           # (N, D)

    return X_prime


In [13]:
# Copy data into matrix
X = np.zeros((N, D))
X[0, :] = np.squeeze(all_x[0])
X[1, :] = np.squeeze(all_x[1])
X[2, :] = np.squeeze(all_x[2])


# Run the self attention mechanism
X_prime = self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Print out the results
print(X_prime)

[[ 0.43250036  1.25670068  0.6858925  -2.0111436 ]
 [ 0.00828235 -1.08920945  0.68373683 -1.37055217]
 [-0.11939362 -1.92678975  0.73672173 -1.46037153]]


If you did this correctly, the values should be the same as above.

TODO:  

Print out the attention matrix
You will see that the values are quite extreme (one is very close to one and the others are very close to zero.  Now we'll fix this problem by using scaled dot-product attention.

In [14]:
def scaled_dot_product_self_attention(X, omega_v, omega_q, omega_k, beta_v, beta_q, beta_k):

    # 1. Compute Q, K, V
    Q = X @ omega_q + beta_q.T      # (N, D)
    K = X @ omega_k + beta_k.T      # (N, D)
    V = X @ omega_v + beta_v.T      # (N, D)

    # 2. Compute dot products (similarity matrix)
    scores = Q @ K.T                # (N, N)

    # 3. Scale by sqrt(D)
    D = X.shape[1]
    scores = scores / np.sqrt(D)

    # 4. Softmax row-wise to obtain attention weights
    scores = scores - np.max(scores, axis=1, keepdims=True)  # numerical stability
    attention = np.exp(scores) / np.sum(np.exp(scores), axis=1, keepdims=True)

    # 5. Compute weighted values
    X_prime = attention @ V         # (N, D)

    return X_prime


In [15]:
# Run the self attention mechanism
X_prime = scaled_dot_product_self_attention(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k)

# Print out the results
print(X_prime)

[[ 0.40797669  1.18303771  0.66050715 -1.84100293]
 [ 0.08863953 -0.60075218  0.66616966 -1.39717579]
 [-0.04331903 -1.43801028  0.70934494 -1.42895216]]


TODO -- Investigate whether the self-attention mechanism is covariant with respect to permutation.
If it is, when we permute the columns of the input matrix $\mathbf{X}$, the columns of the output matrix $\mathbf{X}'$ will also be permuted.
